This notebook performs a fuzzy match on single-cells from the profile data to the endpoint data.
This is necessary because  the endpoint data was not included in the tracking module.
Further this notebook provides information about how long the cell track is.

In [1]:
import pathlib
import time

import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean

try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False
if in_notebook:
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

In [2]:
sc_profile_file_path = pathlib.Path(
    "../../data/CP_scDINO_features/combined_CP_scDINO_norm_fs.parquet"
).resolve(strict=True)
endpoint_sc_profile_file_path = pathlib.Path(
    "../../data/CP_feature_select/endpoints/features_selected_profile.parquet"
).resolve(strict=True)
sc_profile_df = pd.read_parquet(sc_profile_file_path)
endpoint_sc_profile_df = pd.read_parquet(endpoint_sc_profile_file_path)
endpoint_sc_profile_df["Metadata_Well_FOV"] = (
    endpoint_sc_profile_df["Metadata_Well"].astype(str)
    + "_"
    + endpoint_sc_profile_df["Metadata_FOV"].astype(str)
)
endpoint_sc_profile_df["Metadata_sc_unique_track_id"] = (
    endpoint_sc_profile_df["Metadata_Well"].astype(str)
    + "_"
    + endpoint_sc_profile_df["Metadata_FOV"].astype(str)
    + "_"
    + endpoint_sc_profile_df["Metadata_track_id"].astype(str)
)
print(endpoint_sc_profile_df.shape)
sc_profile_df["Metadata_sc_unique_track_id"] = (
    sc_profile_df["Metadata_Well"].astype(str)
    + "_"
    + sc_profile_df["Metadata_FOV"].astype(str)
    + "_"
    + sc_profile_df["Metadata_track_id"].astype(str)
)
sc_profile_df["Metadata_Well_FOV"] = (
    sc_profile_df["Metadata_Well"].astype(str)
    + "_"
    + sc_profile_df["Metadata_FOV"].astype(str)
)
print(sc_profile_df.shape)
sc_profile_df.head()

(64656, 539)
(142040, 2427)


,Metadata_plate,Metadata_Well,Metadata_number_of_singlecells,Metadata_compound,Metadata_dose,Metadata_control,Metadata_ImageNumber,Metadata_FOV,Metadata_Time,Metadata_Cells_Number_Object_Number,...,channel_DNA_cls_feature_93_scDINO,channel_DNA_cls_feature_94_scDINO,channel_DNA_cls_feature_95_scDINO,channel_DNA_cls_feature_96_scDINO,channel_DNA_cls_feature_97_scDINO,channel_DNA_cls_feature_98_scDINO,channel_DNA_cls_feature_99_scDINO,channel_DNA_cls_feature_9_scDINO,Metadata_sc_unique_track_id,Metadata_Well_FOV
0,1,C-10,150,Staurosporine,78.13,test,11,0003,10.0,1.0,...,1.649449,0.010330,0.426311,-0.339320,1.586745,-0.362606,0.811210,1.215151,C-10_0003_2,C-10_0003
1,1,C-10,150,Staurosporine,78.13,test,11,0003,10.0,15.0,...,-1.417577,0.262575,1.097736,-1.700848,0.563939,0.238832,0.717390,0.768155,C-10_0003_3,C-10_0003
2,1,C-10,150,Staurosporine,78.13,test,11,0003,10.0,16.0,...,-0.678053,1.008923,-0.144754,-1.514908,-0.609500,0.447918,1.357684,1.000216,C-10_0003_144,C-10_0003
3,1,C-10,150,Staurosporine,78.13,test,11,0003,10.0,26.0,...,-0.939257,-0.584025,2.944528,-0.227416,0.076889,-1.431668,-0.097650,0.970673,C-10_0003_6,C-10_0003
4,1,C-10,150,Staurosporine,78.13,test,11,0003,10.0,37.0,...,2.372842,-0.591197,-0.304200,0.505422,-0.360512,1.295638,1.948911,0.107559,C-10_0003_21,C-10_0003


In [3]:
pd.set_option("display.max_columns", None)
sc_profile_df

Metadata_plate Metadata_Well  Metadata_number_of_singlecells  \
0                    1          C-10                             150   
1                    1          C-10                             150   
2                    1          C-10                             150   
3                    1          C-10                             150   
4                    1          C-10                             150   
...                ...           ...                             ...   
142035               1          E-05                             193   
142036               1          E-05                             193   
142037               1          E-05                             193   
142038               1          E-05                             193   
142039               1          E-05                             193   

       Metadata_compound Metadata_dose Metadata_control Metadata_ImageNumber  \
0          Staurosporine         78.13             test                   11   
1          Staurosporine         78.13             test                   11   
2          Staurosporine         78.13             test                   11   
3          Staurosporine         78.13             test                   11   
4          Staurosporine         78.13             test                   11   
...                  ...           ...              ...                  ...   
142035     Staurosporine          2.44             test                    2   
142036     Staurosporine          2.44             test                    2   
142037     Staurosporine          2.44             test                    2   
142038     Staurosporine          2.44             test                    2   
142039     Staurosporine          2.44             test                    2   

       Metadata_FOV Metadata_Time  Metadata_Cells_Number_Object_Number  \
0              0003          10.0                                  1.0   
1              0003          10.0                                 15.0   
2              0003          10.0                                 16.0   
3              0003          10.0                                 26.0   
4              0003          10.0                                 37.0   
...             ...           ...                                  ...   
142035         0002           1.0                                180.0   
142036         0002           1.0                                181.0   
142037         0002           1.0                                183.0   
142038         0002           1.0                                185.0   
142039         0002           1.0                                186.0   

        Metadata_Cytoplasm_Parent_Cells  Metadata_Cytoplasm_Parent_Nuclei  \
0                                     1                                 5   
1                                    15                                13   
2                                    16                                 7   
3                                    26                                18   
4                                    37                                32   
...                                 ...                               ...   
142035                              180                               179   
142036                              181                               181   
142037                              183                               183   
142038                              185                               180   
142039                              186                               184   

        Metadata_ImageNumber_1  Metadata_ImageNumber_2  \
0                           11                    11.0   
1                           11                    11.0   
2                           11                    11.0   
3                           11                    11.0   
4                           11                    11.0   
...                 

In [4]:
# drop all nan values in the location columns
endpoint_sc_profile_df = endpoint_sc_profile_df.dropna(
    subset=["Metadata_Nuclei_Location_Center_X", "Metadata_Nuclei_Location_Center_Y"]
)
# sort by Metadata_Well_FOV
endpoint_sc_profile_df = endpoint_sc_profile_df.sort_values(
    by=["Metadata_Well_FOV"]
).reset_index(drop=True)

In [5]:
last_time_point_df = sc_profile_df.loc[
    sc_profile_df["Metadata_Well_FOV"].isin(
        endpoint_sc_profile_df["Metadata_Well_FOV"].unique()
    )
]
print(last_time_point_df.shape)
last_time_point_df = last_time_point_df.loc[
    last_time_point_df["Metadata_Time"] == last_time_point_df["Metadata_Time"].max()
]
print(last_time_point_df.shape)

(138831, 2427)
(10248, 2427)


In [6]:
last_time_point_df["Metadata_id"].unique().shape[0],

(193,)

In [7]:
last_time_point_df["Metadata_track_id"].unique().shape[0],

(224,)

In [8]:
last_time_point_df["Metadata_sc_unique_track_id"].unique().shape[0]

9948

In [9]:
# remove any rows with NaN values in the location columns
last_time_point_df = last_time_point_df.dropna(
    subset=["Metadata_Nuclei_Location_Center_X", "Metadata_Nuclei_Location_Center_Y"]
)
endpoint_sc_profile_df = endpoint_sc_profile_df.dropna(
    subset=["Metadata_Nuclei_Location_Center_X", "Metadata_Nuclei_Location_Center_Y"]
)
print(endpoint_sc_profile_df.shape)
print(last_time_point_df.shape)

(64656, 539)
(10248, 2427)


In [10]:
pd.set_option("display.max_columns", None)

In [11]:
# drop the nan values
print(last_time_point_df.shape)
last_time_point_df = last_time_point_df.dropna(how="any")
print(last_time_point_df.shape)

(10248, 2427)
(10248, 2427)


In [16]:
# sort each dataframe by Metadata_Well_FOV
last_time_point_df = last_time_point_df.sort_values(
    by=["Metadata_Well_FOV"]
).reset_index(drop=True)
endpoint_sc_profile_df = endpoint_sc_profile_df.sort_values(
    by=["Metadata_Well_FOV"]
).reset_index(drop=True)

In [30]:
for i, row in last_time_point_df.iterrows():
    for j, row2 in endpoint_sc_profile_df.iterrows():
        if row["Metadata_Well_FOV"] == row2["Metadata_Well_FOV"]:
            # calculate the euclidean distance between the two points
            dist = euclidean(
                (
                    row["Metadata_Nuclei_Location_Center_X"],
                    row["Metadata_Nuclei_Location_Center_Y"],
                ),
                (
                    row2["Metadata_Nuclei_Location_Center_X"],
                    row2["Metadata_Nuclei_Location_Center_Y"],
                ),
            )
            if dist < 5:
                print(dist)

3.162733307498641
3.162733307498641
3.162733307498641
3.162733307498641
3.162733307498641
3.162733307498641
3.162733307498641
3.162733307498641
3.162733307498641
3.162733307498641
3.162733307498641
3.162733307498641
1.5419986379459583
1.5419986379459583
1.5419986379459583
1.5419986379459583
1.5419986379459583
1.5419986379459583
1.5419986379459583
1.5419986379459583
1.5419986379459583
1.5419986379459583
1.5419986379459583
1.5419986379459583
4.686828960669876
4.686828960669876
4.686828960669876
4.686828960669876
4.686828960669876
4.686828960669876
4.686828960669876
4.686828960669876
4.686828960669876
4.686828960669876
4.686828960669876
4.686828960669876


KeyboardInterrupt: 

In [17]:
# chunk the dataframe by well_fov so that there are fewer indexed records to search at once during fuzzy matching
dict_of_sc_well_fovs = {}
for well_fov in last_time_point_df["Metadata_Well_FOV"].unique()[:1]:
    dict_of_sc_well_fovs[well_fov] = last_time_point_df[
        last_time_point_df["Metadata_Well_FOV"] == well_fov
    ].copy()
    dict_of_sc_well_fovs[well_fov].reset_index(drop=True, inplace=True)
    # make a distance column for each well_fov
dict_of_sc_well_fovs_endpoint = {}
for well_fov in endpoint_sc_profile_df["Metadata_Well_FOV"].unique()[:1]:
    dict_of_sc_well_fovs_endpoint[well_fov] = endpoint_sc_profile_df[
        endpoint_sc_profile_df["Metadata_Well_FOV"] == well_fov
    ].copy()
    dict_of_sc_well_fovs_endpoint[well_fov].reset_index(drop=True, inplace=True)
    # make a distance column for each well_fov
    dict_of_sc_well_fovs_endpoint[well_fov]["distance"] = np.nan

In [18]:
start_time = time.time()

In [28]:
for well_fov in tqdm(list(dict_of_sc_well_fovs.keys()), desc="Processing Well-FOVs"):
    for i, row in tqdm(
        dict_of_sc_well_fovs[well_fov].iterrows(),
        total=len(dict_of_sc_well_fovs[well_fov]),
        desc="Outer Loop",
        leave=False,
    ):
        for j, row2 in dict_of_sc_well_fovs_endpoint[well_fov].iterrows():
            # check that the well_fov is the same
            if row["Metadata_Well_FOV"] == row2["Metadata_Well_FOV"]:
                distance = abs(
                    euclidean(
                        [
                            row["Metadata_Nuclei_Location_Center_X"],
                            row["Metadata_Nuclei_Location_Center_Y"],
                        ],
                        [
                            row2["Metadata_Nuclei_Location_Center_X"],
                            row2["Metadata_Nuclei_Location_Center_Y"],
                        ],
                    )
                )
                if distance < 5:
                    print(distance)
                    if (
                        dict_of_sc_well_fovs_endpoint[well_fov].at[j, "distance"]
                        is np.nan
                    ):
                        dict_of_sc_well_fovs_endpoint[well_fov].at[
                            j, "distance"
                        ] = distance
                        dict_of_sc_well_fovs_endpoint[well_fov].at[
                            j, "Metadata_sc_unique_track_id"
                        ] = row["Metadata_sc_unique_track_id"]

                    elif (
                        dict_of_sc_well_fovs_endpoint[well_fov].at[j, "distance"]
                        > distance
                    ):
                        dict_of_sc_well_fovs_endpoint[well_fov].at[
                            j, "distance"
                        ] = distance
                        dict_of_sc_well_fovs_endpoint[well_fov].at[
                            j, "Metadata_sc_unique_track_id"
                        ] = row["Metadata_sc_unique_track_id"]
                        print(distance)

Processing Well-FOVs:   0%|          | 0/1 [00:00<?, ?it/s]

Outer Loop:   0%|          | 0/114 [00:00<?, ?it/s]

3.162733307498641
3.162733307498641
3.162733307498641
3.162733307498641
3.162733307498641
3.162733307498641
3.162733307498641
3.162733307498641
3.162733307498641
3.162733307498641
3.162733307498641
3.162733307498641
1.5419986379459583
1.5419986379459583
1.5419986379459583
1.5419986379459583
1.5419986379459583
1.5419986379459583
1.5419986379459583
1.5419986379459583
1.5419986379459583
1.5419986379459583
1.5419986379459583
1.5419986379459583


In [20]:
print("Fuzzy matching completed!")
print(f"Took {time.time() - start_time} seconds")
print(f"Took {round((time.time() - start_time) / 60, 2)} minutes")
print(f"Took {round((time.time() - start_time) / 3600, 2)} hours")

Fuzzy matching completed!
Took 10.9288489818573 seconds
Took 0.18 minutes
Took 0.0 hours


In [21]:
sc_well_fovs_endpoint_df = pd.concat(
    dict_of_sc_well_fovs_endpoint.values(), ignore_index=True
)
sc_well_fovs_endpoint_df["distance"].unique()

array([nan])

In [22]:
# drop the rows where Metadata_sc_unique_track_id is NaN
sc_well_fovs_endpoint_df = sc_well_fovs_endpoint_df.dropna(
    subset=["Metadata_sc_unique_track_id"]
)
print(sc_well_fovs_endpoint_df.shape)
sc_well_fovs_endpoint_df.reset_index(drop=True, inplace=True)
sc_well_fovs_endpoint_df["Metadata_Time"] = 13.0
sc_well_fovs_endpoint_df.head()

(1320, 540)


,Metadata_plate,Metadata_Well,Metadata_number_of_singlecells,Metadata_compound,Metadata_dose,Metadata_control,Metadata_ImageNumber,Metadata_FOV,Metadata_Time,Metadata_Cells_Number_Object_Number,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_ImageNumber_1,Metadata_ImageNumber_2,Metadata_ImageNumber_3,Metadata_Nuclei_Number_Object_Number,Metadata_Image_FileName_AnnexinV,Metadata_Image_FileName_DNA,Metadata_Image_PathName_AnnexinV,Metadata_Image_PathName_DNA,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_coordinates_x,Metadata_track_id,Metadata_t,Metadata_y,Metadata_x,Metadata_id,Metadata_parent_track_id,Metadata_parent_id,Metadata_coordinates_y,Metadata_distance,Cytoplasm_AreaShape_Area,Cytoplasm_AreaShape_BoundingBoxMaximum_Y,Cytoplasm_AreaShape_Compactness,Cytoplasm_AreaShape_Eccentricity,Cytoplasm_AreaShape_Extent,Cytoplasm_AreaShape_FormFactor,Cytoplasm_AreaShape_MaximumRadius,Cytoplasm_AreaShape_MinorAxisLength,Cytoplasm_AreaShape_Orientation,Cytoplasm_AreaShape_Solidity,Cytoplasm_AreaShape_Zernike_0_0,Cytoplasm_AreaShape_Zernike_1_1,Cytoplasm_AreaShape_Zernike_2_0,Cytoplasm_AreaShape_Zernike_2_2,Cytoplasm_AreaShape_Zernike_3_1,Cytoplasm_AreaShape_Zernike_3_3,Cytoplasm_AreaShape_Zernike_4_0,Cytoplasm_AreaShape_Zernike_5_1,Cytoplasm_AreaShape_Zernike_5_3,Cytoplasm_AreaShape_Zernike_5_5,Cytoplasm_AreaShape_Zernike_6_0,Cytoplasm_AreaShape_Zernike_7_1,Cytoplasm_AreaShape_Zernike_7_3,Cytoplasm_AreaShape_Zernike_7_5,Cytoplasm_AreaShape_Zernike_7_7,Cytoplasm_AreaShape_Zernike_9_1,Cytoplasm_AreaShape_Zernike_9_3,Cytoplasm_AreaShape_Zernike_9_5,Cytoplasm_AreaShape_Zernike_9_7,Cytoplasm_AreaShape_Zernike_9_9,Cytoplasm_Correlation_Correlation_AnnexinV_DNA,Cytoplasm_Correlation_K_AnnexinV_DNA,Cytoplasm_Correlation_K_DNA_AnnexinV,Cytoplasm_Correlation_Manders_AnnexinV_DNA,Cytoplasm_Correlation_Manders_DNA_AnnexinV,Cytoplasm_Correlation_Overlap_AnnexinV_DNA,Cytoplasm_Granularity_1_AnnexinV,Cytoplasm_Granularity_1_DNA,Cytoplasm_Intensity_IntegratedIntensityEdge_DNA,Cytoplasm_Intensity_IntegratedIntensity_AnnexinV,Cytoplasm_Intensity_IntegratedIntensity_DNA,Cytoplasm_Intensity_MassDisplacement_AnnexinV,Cytoplasm_Intensity_MassDisplacement_DNA,Cytoplasm_Intensity_MaxIntensityEdge_AnnexinV,Cytoplasm_Intensity_MaxIntensity_AnnexinV,Cytoplasm_Intensity_MeanIntensityEdge_DNA,Cytoplasm_Intensity_StdIntensityEdge_DNA,Cytoplasm_Intensity_StdIntensity_DNA,Cytoplasm_RadialDistribution_FracAtD_AnnexinV_3of4,Cytoplasm_RadialDistribution_FracAtD_AnnexinV_4of4,Cytoplasm_RadialDistribution_FracAtD_DNA_1of4,Cytoplasm_RadialDistribution_FracAtD_DNA_2of4,Cytoplasm_RadialDistribution_FracAtD_DNA_3of4,Cytoplasm_RadialDistribution_FracAtD_DNA_4of4,Cytoplasm_RadialDistribution_MeanFrac_AnnexinV_3of4,Cytoplasm_RadialDistribution_MeanFrac_AnnexinV_4of4,Cytoplasm_RadialDistribution_MeanFrac_DNA_2of4,Cytoplasm_RadialDistribution_MeanFrac_DNA_3of4,Cytoplasm_RadialDistribution_MeanFrac_DNA_4of4,Cytoplasm_RadialDistribution_RadialCV_AnnexinV_1of4,Cytoplasm_RadialDistribution_RadialCV_AnnexinV_2of4,Cytoplasm_RadialDistribution_RadialCV_AnnexinV_3of4,Cytoplasm_RadialDistribution_RadialCV_AnnexinV_4of4,Cytoplasm_RadialDistribution_RadialCV_DNA_1of4,Cytoplasm_RadialDistribution_RadialCV_DNA_2of4,Cytoplasm_RadialDistribution_RadialCV_DNA_3of4,Cytoplasm_RadialDistribution_RadialCV_DNA_4of4,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_2_0,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_4_0,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_4_2,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_4_4,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_6_0,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_7_1,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_7_5,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_7_7,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_8_6,Cytoplasm_RadialDistribution_ZernikeMagnitude_AnnexinV_8_8,Cytoplasm_RadialDistribution_ZernikeMagnitude_Anne

In [ ]:
# map the value counts to a new column for each Metadata_sc_unique_track_id
sc_profile_df["Metadata_sc_unique_track_id_count"] = sc_profile_df[
    "Metadata_sc_unique_track_id"
].map(sc_profile_df["Metadata_sc_unique_track_id"].value_counts())
sc_profile_df.head()

In [ ]:
# write the cleaned dataframe to a parquet file
output_sc_file_path = pathlib.Path("../results/cleaned_sc_profile.parquet").resolve(
    strict=False
)
output_sc_endpoint_file_path = pathlib.Path(
    "../results/cleaned_endpoint_sc_profile.parquet"
).resolve(strict=False)
output_sc_file_path.parent.mkdir(parents=True, exist_ok=True)

# we save the two profiles separately because they have different feature spaces

sc_profile_df.to_parquet(output_sc_file_path, index=False)
sc_well_fovs_endpoint_df.to_parquet(output_sc_endpoint_file_path, index=False)